In [2]:
from tqdm import tqdm

from utils.elt_config import (
    DataDirectory,
    create_all_schemas,
    get_all_schemas,
    get_cnxn,
    get_info_schema_df,
)

In [3]:
cnxn = get_cnxn(database=":memory:")
create_all_schemas(cnxn)

In [4]:
data_dir = DataDirectory()
data_dir.queries.load_landing_data(cnxn)

In [6]:
info_schema_df = get_info_schema_df(cnxn)
schemas = get_all_schemas(info_schema_df)
print("schemas:", schemas)
info_schema_df.loc[:, ["schema", "name"]].head(20)

schemas: ['landing']


,schema,name
0,landing,assessments
1,landing,courses
2,landing,student_assessment
3,landing,student_info
4,landing,student_registration
5,landing,student_vle
6,landing,vle


In [12]:
queries = data_dir.queries

for sql_file in tqdm(
    sorted(
        [x for x in queries.allchildfiles if x.path.suffix == ".sql"],
        key=lambda x: str(x.path.parent),
    )
):
    sql = sql_file.path.read_text()
    upper = sql.upper()
    for cmd in ["CREATE", "DROP", "INTO"]:
        if cmd in upper:
            raise ValueError(f"{cmd} in SQL {sql_file.path}")
    stem_parts = sql_file.path.stem.split("_")
    if len(stem_parts) < 2:
        raise ValueError(f"stem {sql_file.path.stem} has less than 2 parts")
    schema = stem_parts[0]
    if schema not in schemas:
        raise ValueError(f"schema {schema} not in schemas {schemas}")
    table = "_".join(stem_parts[1:])

 10%|█         | 10/100 [00:00<00:00, 1730.11it/s]


ValueError: CREATE in SQL /Users/Fr333y3d3a/repos/master_proj/src/queries/01_main/01_dimensions/final_result.sql